In [93]:
from pathlib import Path

import numpy as np
import pandas as pd


In [94]:
input_data = Path("example.txt").read_text().splitlines()
print(input_data)

['11-22,95-115,998-1012,1188511880-1188511890,222220-222224,1698522-1698528,446443-446449,38593856-38593862,565653-565659,824824821-824824827,2121212118-2121212124']


In [95]:
id_ranges = [line.split("-") for line in input_data[0].split(",")]
id_ranges

[['11', '22'],
 ['95', '115'],
 ['998', '1012'],
 ['1188511880', '1188511890'],
 ['222220', '222224'],
 ['1698522', '1698528'],
 ['446443', '446449'],
 ['38593856', '38593862'],
 ['565653', '565659'],
 ['824824821', '824824827'],
 ['2121212118', '2121212124']]

In [96]:
id_range_df = (
    pd.DataFrame(id_ranges, columns=["first_id", "last_id"])
    .astype(
        int,
    )
    .sort_values("first_id")
    .assign(
        id_hash=lambda df: df.apply(
            lambda row: hash((row.first_id, row.last_id)),
            axis=1,
        ),
    )
)
id_range_df

,first_id,last_id,id_hash
0,11,22,6591627001642629966
1,95,115,6327338302465558770
2,998,1012,7551883741941015371
4,222220,222224,-7323056831730286046
6,446443,446449,-6680421239719204760
8,565653,565659,2768759020111293564
5,1698522,1698528,-3099262812073348782
7,38593856,38593862,7445099129516273786
9,824824821,824824827,4037462719799829074
3,1188511880,1188511890,7481919396115548520


## Part I

In [97]:
min_id = id_range_df.first_id.min()
max_id = id_range_df.last_id.max()


def double(value: int) -> int:
    value_str = str(value)
    return int(value_str + value_str)


value = 1
checks = []
while double(value) <= max_id:
    checks.append(double(value))
    value += 1
check_df = pd.DataFrame(checks, columns=["value"])
print(check_df.shape[0])
check_df.head()
check_df.tail()

21211


,value
21206,2120721207
21207,2120821208
21208,2120921209
21209,2121021210
21210,2121121211


In [98]:
invalid_df = pd.merge_asof(
    left=check_df,
    right=id_range_df.rename(columns={"id_hash": "id_hash_1"})[
        ["first_id", "id_hash_1"]
    ],
    left_on="value",
    right_on="first_id",
    direction="backward",
)

invalid_df = pd.merge_asof(
    left=invalid_df,
    right=id_range_df.rename(columns={"id_hash": "id_hash_2"})[
        ["last_id", "id_hash_2"]
    ],
    left_on="value",
    right_on="last_id",
    direction="forward",
)
invalid_df = invalid_df.assign(
    is_valid=lambda df: df.id_hash_1 != df.id_hash_2,
)

invalid_df

,value,first_id,id_hash_1,last_id,id_hash_2,is_valid
0,11,11,6591627001642629966,22,6591627001642629966,False
1,22,11,6591627001642629966,22,6591627001642629966,False
2,33,11,6591627001642629966,115,6327338302465558770,True
3,44,11,6591627001642629966,115,6327338302465558770,True
4,55,11,6591627001642629966,115,6327338302465558770,True
...,...,...,...,...,...,...
21206,2120721207,1188511880,7481919396115548520,2121212124,9180051630190218805,True
21207,2120821208,1188511880,7481919396115548520,2121212124,9180051630190218805,True
21208,2120921209,1188511880,7481919396115548520,2121212124,9180051630190218805,True
21209,2121021210,1188511880,7481919396115548520,2121212124,9180051630190218805,True


In [99]:
invalid_df.loc[~invalid_df.is_valid, "value"].sum()

np.int64(1227775554)

## Part II

In [100]:
min_id = id_range_df.first_id.min()
max_id = id_range_df.last_id.max()


def multiply(value: int) -> list[int]:
    value_str = str(value)
    return [
        int(value_str * n)
        for n in range(2, len(str(max_id)))
        if int(value_str * n) <= max_id
    ]


value = 1
checks = []
while len(str(value)) < len(str(max_id)) // 2 + 1:
    checks += multiply(value)
    value += 1

check_df = (
    pd.DataFrame(checks, columns=["value"], dtype=np.int64)
    .drop_duplicates()
    .sort_values("value")
)
print(check_df.shape[0])
check_df.head()

22230


,value
0,11
8,22
16,33
24,44
32,55


In [101]:
invalid_df = pd.merge_asof(
    left=check_df,
    right=id_range_df.rename(columns={"id_hash": "id_hash_1"})[
        ["first_id", "id_hash_1"]
    ],
    left_on="value",
    right_on="first_id",
    direction="backward",
)

invalid_df = pd.merge_asof(
    left=invalid_df,
    right=id_range_df.rename(columns={"id_hash": "id_hash_2"})[
        ["last_id", "id_hash_2"]
    ],
    left_on="value",
    right_on="last_id",
    direction="forward",
)
invalid_df = invalid_df.assign(
    is_valid=lambda df: df.id_hash_1 != df.id_hash_2,
)

invalid_df


,value,first_id,id_hash_1,last_id,id_hash_2,is_valid
0,11,11,6591627001642629966,22,6591627001642629966,False
1,22,11,6591627001642629966,22,6591627001642629966,False
2,33,11,6591627001642629966,115,6327338302465558770,True
3,44,11,6591627001642629966,115,6327338302465558770,True
4,55,11,6591627001642629966,115,6327338302465558770,True
...,...,...,...,...,...,...
22225,2120821208,1188511880,7481919396115548520,2121212124,9180051630190218805,True
22226,2120921209,1188511880,7481919396115548520,2121212124,9180051630190218805,True
22227,2121021210,1188511880,7481919396115548520,2121212124,9180051630190218805,True
22228,2121121211,1188511880,7481919396115548520,2121212124,9180051630190218805,True


In [102]:
invalid_df.loc[~invalid_df.is_valid, "value"].sum()


np.int64(4174379265)